Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 12.1 : Weather	Pattern	Clustering using k-mean in Spark
Learning	Goals

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_en

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/

#pwd
!ls

In [ ]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/Data-Cluster

#pwd
!ls

In [ ]:
!cp utils.py /content/utils.py

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
import utils

%matplotlib inline

In [ ]:
sqlsc = SQLContext(sc)

df = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/Data-Cluster/minute_weather.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')


***Subset and remove unused data***

In [ ]:
df.count()

***Question 1.***

---



What percentage of samples have 0 for rain_accumulation?

A. 157812 / 158726 = 99.4%

B. 157237 / 158726 = 99.1%

C. There is not enough information to determine this D. None of the above

ANS: A

In [ ]:
filteredDf = df.filter((df.rowID % 5) == 0) # 10th, 2oth, 30th
filteredDf.count()

***Q3***


---


If we wanted to create a data subset by taking every 5th sample instead of every 10th sample, how many samples would be in that subset?

A. 317,452

B. 1,587,257

C. 158,726

D. None of the above

ANS: A



In [ ]:
filteredDf.describe().toPandas().transpose()

In [ ]:
filteredDf.filter(filteredDf.rain_accumulation == 0.0).count()

In [ ]:
filteredDf.filter(filteredDf.rain_duration == 0.0).count()

In [ ]:
workingDf = filteredDf.drop("rain_accumulation").drop("rain_duration"). drop("hpwren_timestamp")

In [ ]:
before = workingDf.count()
workingDf = workingDf.na.drop()
after = workingDf.count()
before - after

***Scale the data***

In [ ]:
workingDf.columns

In [ ]:
featuresUsed = [
 'air_pressure',
 'air_temp',
 'avg_wind_direction',
 'avg_wind_speed',
 'max_wind_direction',
 'max_wind_speed',
 'relative_humidity']

assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(workingDf)

In [ ]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

***Create elbow plot***

In [ ]:
scaledData = scaledData.select("features", "rowID")
elbowset = scaledData.filter((scaledData.rowID % 3) == 0).select("features")
elbowset.persist()

In [ ]:
clusters = range(2,31)
wsseList = utils.elbow(elbowset, clusters)

In [ ]:
utils.elbow_plot(wsseList, clusters)

***Cluster using selected K***

In [ ]:
scaledDataFeat = scaledData.select("features")
scaledDataFeat.persist()

***Q7.***


---


If we perform clustering with 20 clusters (and seed = 1), which cluster appears to identify Santa Ana conditions (lowest humidity and highest wind speeds)?




A. Cluster 12

B. Cluster 1

C. Cluster 16

D. None of the above

ANS A

In [ ]:
kmeans = KMeans(k=20, seed=1)
model = kmeans.fit(scaledDataFeat)
transformed = model.transform(scaledDataFeat)

In [ ]:
centers = model.clusterCenters()
centers

***Create parallel plots of clusters and analysis***

In [ ]:
P = utils.pd_centers(featuresUsed, centers)

In [ ]:
utils.parallel_plot(P[P['relative_humidity'] < -0.5], P)

In [ ]:
utils.parallel_plot(P[P['air_temp'] > 0.5], P)

In [ ]:
utils.parallel_plot(P[(P['relative_humidity'] > 0.5) & (P['air_temp'] < 0.5)] , P)

***Other days***

In [ ]:
utils.parallel_plot(P.iloc[[1,3]], P)

In [ ]:
utils.parallel_plot(P.iloc[[7,8,11]], P)

***Q8.***



---


We did not include the minimum wind measurements in the analysis since they are highly correlated with the average wind measurements. What is the correlation between min_wind_speed and avg_wind_speed (to two decimals)? (Compute this using one-tenth of the original dataset, and dropping all rows with missing values.)

A. 0.97

B. -0.12

C. 0.62

D. None of the above

ANS: A

In [ ]:
filteredDf = df.filter((df.rowID % 10) == 0) # 10th, 2oth, 30th
filteredDf.count()


filteredDf.describe().toPandas().transpose()

filteredDf.filter(filteredDf.rain_accumulation == 0.0).count()

filteredDf.filter(filteredDf.rain_duration == 0.0).count()

workingDf = filteredDf.drop("rain_accumulation").drop("rain_duration"). drop("hpwren_timestamp")

before = workingDf.count()
workingDf = workingDf.na.drop()
after = workingDf.count()
before - after


workingDf.columns

featuresUsed = [
 'air_pressure',
 'air_temp',
 'avg_wind_direction',
 'avg_wind_speed',
 'max_wind_direction',
 'max_wind_speed',
 'relative_humidity']

assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(workingDf)

scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)


scaledData = scaledData.select("features", "rowID")
elbowset = scaledData.filter((scaledData.rowID % 3) == 0).select("features")
elbowset.persist()

clusters = range(2,31)
wsseList = utils.elbow(elbowset, clusters)

utils.elbow_plot(wsseList, clusters)


scaledDataFeat = scaledData.select("features")
scaledDataFeat.persist()




P = utils.pd_centers(featuresUsed, centers)

utils.parallel_plot(P[P['relative_humidity'] < -0.5], P)

utils.parallel_plot(P[P['air_temp'] > 0.5], P)

utils.parallel_plot(P[(P['relative_humidity'] > 0.5) & (P['air_temp'] < 0.5)] , P)


utils.parallel_plot(P.iloc[[1,3]], P)

utils.parallel_plot(P.iloc[[7,8,11]], P)
